# Download Dataset

In [ ]:
import urllib.request

url = "https://www.dropbox.com/scl/fo/5w0a14icfv4o7t0azrqda/AHECd04T6OAUwcvxwiZXw-4/data/tmall?rlkey=qhx7csgahlcbuppjx4ewa3l0o&subfolder_nav_tracking=1&st=kqkod5je&dl=1"
output_path = "tmall.zip"

print("Downloading...")
urllib.request.urlretrieve(url, output_path)
print("Download complete!")

In [ ]:
import zipfile as zip
import os

with zip.ZipFile(output_path, 'r') as zip_ref:
    zip_ref.extractall("tmall_data")
print("Extraction complete!")

In [2]:
import pandas as pd
df_nodes = pd.read_csv("tmall_data/node2label.txt", sep=r'\s+', header=None)

In [3]:
df_nodes.columns=['user','product']

df_nodes.head()
df_nodes.shape
df_nodes.nunique()

user       81380
product        5
dtype: int64

In [4]:
df_edges = pd.read_csv("tmall_data/tmall.txt", sep=r'\s+', header=None)

In [5]:
df_edges.columns=['source_node','target_node','weight']

df_edges.head()
df_edges.shape
df_edges.nunique()

source_node    100000
target_node    477314
weight            186
dtype: int64

# Implement GNN and GCN using torch geometric

**Preparing dataset**

In [6]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


**Convert edges**

In [7]:
import torch.nn as nn
from torch_geometric.data import Data

edge_index=torch.tensor(df_edges[['source_node','target_node']].values.T, dtype=torch.long)
edge_weight=torch.tensor(df_edges['weight'].values, dtype=torch.float)

num_nodes = max(df_edges['source_node'].max(), df_edges['target_node'].max()) +1

c:\Users\S.Soudeep\Documents\Contrastive_Spiking_Graph_NeuralNetwork\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
embedding_dim = 64
node_embeddings = nn.Embedding(num_nodes, embedding_dim)
x=node_embeddings.weight

data = Data(x=x, edge_index=edge_index, edge_attr=edge_weight)

num_nodes = data.num_nodes

peram = torch.randperm(num_nodes)
train_size = int(0.6*num_nodes)
val_size = int(0.2*num_nodes)

In [10]:
data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.val_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)


data.train_mask[peram[:train_size]] = True
data.val_mask[peram[train_size:train_size+val_size]] = True
data.test_mask[peram[train_size+val_size:]]=True

In [18]:
num_nodes=data.x.shape[0]

data.y = torch.randint(0,2,(num_nodes,), dtype=torch.long)

print("Assigned labels:", data.y[:10])
print("Shape:",data.y.shape)

from torch_geometric.transforms import RandomNodeSplit

transform = RandomNodeSplit(split='random', num_val=0.2, num_test=0.2)
data = transform(data)

Assigned labels: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
Shape: torch.Size([577314])


In [19]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [20]:
import torch.nn.functional as F

CUDA_LAUNCH_BLOCKING=1

model = GCN(hidden_channels=64, in_channels=64, out_channels=2)
data = data

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train_model():
    model.train()
    optimizer.zero_grad()
    output = model(data.x, data.edge_index)
    loss = F.nll_loss(output[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

for epoch in range(200):
    loss_value = train_model()
    print(f'Epoch: {epoch+1:03d}, Loss: {loss_value:.4f}')

RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [21]:
model.eval()
pred = model(data.x, data.edge_index).argmax(dim=1)
correct = pred[data.test_mask] == data.y[data.test_mask]
acc = int(correct.sum()) / int(data.test_mask.sum())
print(f'Test Accuracy: {acc:.4f}')

RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


# Uporer toko done

# Import all the dependecies

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch_geometric.datasets import DBLP
from torch_geometric.data import Data, HeteroData
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import dropout_edge
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedShuffleSplit
from collections import defaultdict
from itertools import combinations
from torch_geometric.utils import k_hop_subgraph
import random

# Preprocessing

In [ ]:
# Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

training_ratios = [0.4, 0.6, 0.8]
num_runs = 5
epochs = 500
hidden_dim = 256
time_steps = 20

In [ ]:
# AFTER (corrected)
edges_file_path = "/kaggle/working/tmall/tmall.txt"
nodes_file_path = "/kaggle/working/tmall/node2label.txt"

In [ ]:
# During node/edge processing
current_idx = 0
node_id_to_idx = {}
labels = []

# 1. Process node2label.txt
with open(nodes_file_path, "r") as f:
    for line in f:
        node_id, label = map(int, line.strip().split())
        if node_id not in node_id_to_idx:
            node_id_to_idx[node_id] = current_idx
            labels.append(label)
            current_idx += 1

# 2. Process edges and add missing nodes
edges = []
with open(edges_file_path, "r") as f:
    for line in f:
        parts = line.strip().split()
        src, dst = map(int, parts[:2])
        
        # Add missing nodes with label -1
        for node in [src, dst]:
            if node not in node_id_to_idx:
                node_id_to_idx[node] = current_idx
                labels.append(-1)
                current_idx += 1
                
        edges.append([node_id_to_idx[src], node_id_to_idx[dst]])

# 3. Final checks
assert len(node_id_to_idx) == len(labels), "Node count mismatch!"
assert max(node_id_to_idx.values()) == len(node_id_to_idx) - 1, "Indexing error"

# 4. Create tensors
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
x = torch.arange(len(node_id_to_idx), dtype=torch.float).view(-1, 1)
y = torch.tensor(labels, dtype=torch.long)
data = Data(x=x, edge_index=edge_index, y=y)

# or:
# data = Data(x=x, edge_index=edge_index, y=y).to(device)

In [ ]:
print(data)

In [ ]:
# Verify node indices in edges
assert data.edge_index.max() < data.num_nodes, \
    f"Edge contains invalid node index {data.edge_index.max()} (num_nodes={data.num_nodes})"

# Check 2: Verify tensor dimensions
print(f"Node features: {data.x.shape}")
print(f"Max node index: {data.edge_index.max().item()}")
print(f"Number of nodes: {data.num_nodes}")

In [ ]:
import numpy as np

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import dropout_edge
import random

torch.manual_seed(42)
random.seed(42)
np.random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ====== ADDED MISSING COMPONENTS ======
class TemporalAttention(nn.Module):
    def __init__(self, time_steps):
        super().__init__()
        self.attention_weights = nn.Parameter(torch.randn(time_steps))
        self.softmax = nn.Softmax(dim=0)

    def forward(self, x):
        # x shape: (time_steps, num_nodes, features)
        alpha = self.softmax(self.attention_weights)
        weighted = alpha.unsqueeze(-1).unsqueeze(-1) * x
        return weighted.sum(dim=0)

class DynamicWeight(nn.Module):
    def __init__(self):
        super().__init__()
        self.alpha = nn.Parameter(torch.tensor([0.5]))

    def forward(self, cls_loss, cont_loss):
        return self.alpha * cls_loss + (1 - self.alpha) * cont_loss
# ======================================

class LIFNeuron(nn.Module):
    def __init__(self, decay=0.95, threshold=1.0):
        super().__init__()
        self.decay = decay
        self.threshold = threshold
        
    def forward(self, x, membrane=None):
        if membrane is None:
            membrane = torch.zeros_like(x)
        
        membrane = self.decay * membrane + x
        spike = torch.sigmoid(5*(membrane - self.threshold))
        
        with torch.no_grad():
            reset = (spike > 0.5).float() * self.threshold
            membrane = membrane - reset
            
        return spike, membrane

class SpikingGCN(nn.Module):
    def __init__(self, in_channels, out_channels, time_steps):
        super().__init__()
        self.conv = GCNConv(in_channels, out_channels)
        self.lif = LIFNeuron()
        self.time_steps = time_steps
        self.temp_attn = TemporalAttention(time_steps)  # Now defined

    def forward(self, x, edge_index):
        x = self.conv(x, edge_index)
        spikes = []
        membrane = None
        
        for _ in range(self.time_steps):
            spike, membrane = self.lif(x, membrane)
            spikes.append(spike)
            
        return self.temp_attn(torch.stack(spikes))

class DyC_DGNN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, time_steps):
        super().__init__()
        self.spike_gcn1 = SpikingGCN(in_channels, hidden_channels, time_steps)
        self.spike_gcn2 = SpikingGCN(hidden_channels, hidden_channels, time_steps)
        
        self.proj_head = nn.Sequential(
            nn.Linear(hidden_channels, 256),
            nn.ReLU(),
            nn.Linear(256, 128)
        )
        
        self.classifier = nn.Linear(hidden_channels, out_channels)
        self.weight_module = DynamicWeight()  # Now defined
        self.temperature = nn.Parameter(torch.tensor(0.1))

    # REQUIRED FORWARD METHOD
    def forward(self, x, edge_index):
        x = F.relu(self.spike_gcn1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        embeddings = self.spike_gcn2(x, edge_index)
        return self.classifier(embeddings), self.proj_head(embeddings)

    def contrastive_loss(self, proj, edge_index, mask):
        masked_nodes = torch.where(mask)[0]
        
        # Create subgraph with proper node relabeling
        subset, sub_edge_index, _, _ = k_hop_subgraph(
            masked_nodes,
            num_hops=2,
            edge_index=edge_index,
            relabel_nodes=True,
            num_nodes=self.x.size(0)
        )
        
        # Get subgraph features
        sub_x = self.x[subset]
        
        # Augment edges
        edge_index1, _ = dropout_edge(sub_edge_index, p=0.4)
        edge_index2, _ = dropout_edge(sub_edge_index, p=0.4)
        
        # Get subgraph projections
        _, proj1 = self.forward(sub_x, edge_index1)
        _, proj2 = self.forward(sub_x, edge_index2)
        
        # Normalize and calculate loss
        proj_sub = F.normalize(proj[subset], dim=1)
        proj1 = F.normalize(proj1, dim=1)
        proj2 = F.normalize(proj2, dim=1)
        
        logits = (proj_sub @ proj1.T) / self.temperature
        labels = torch.arange(proj_sub.size(0), device=proj_sub.device)
        return F.cross_entropy(logits, labels)

    def full_loss(self, logits, y, proj, edge_index, mask):
        cls_loss = F.cross_entropy(logits[mask], y[mask])
        cont_loss = self.contrastive_loss(proj, edge_index, mask)
        return self.weight_module(cls_loss, cont_loss)


In [ ]:
# Modified Training Procedure
def train_model(data, run, ratio):

    SEED = 42 + run  # Different seed for each run but deterministic
    torch.manual_seed(SEED)
    random.seed(SEED)
    np.random.seed(SEED)

    # Stratified splitting
    labels = data.y.cpu().numpy()
    indices = np.arange(len(labels))
    
    sss = StratifiedShuffleSplit(n_splits=1, test_size=1-ratio, random_state=run)
    train_idx, temp_idx = next(sss.split(indices, labels))
    
    sss_temp = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=run)
    val_idx, test_idx = next(sss_temp.split(temp_idx, labels[temp_idx]))
    
    # Create masks
    train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
    
    train_mask[train_idx] = True
    val_mask[temp_idx[val_idx]] = True
    test_mask[temp_idx[test_idx]] = True
    
    data.train_mask = train_mask.to(device)
    data.val_mask = val_mask.to(device)
    data.test_mask = test_mask.to(device)

    hidden_channels=64

    hidden_channels = min(256, data.num_nodes // 50)  # Example adaptive sizing
    model = DyC_DGNN(
        in_channels=data.num_features,
        hidden_channels=hidden_channels,
        out_channels=int(data.y.max()) + 1,
        time_steps=5
    ).to(device)

    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
    
    # Store graph data in model for contrastive learning
    model.x = data.x  # Add this line
    model.edge_index = data.edge_index  # Add this line
    
    # Training loop with dynamic contrastive learning
    best_val_f1 = 0
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        
        logits, proj = model(data.x, data.edge_index)
        loss = model.full_loss(logits, data.y, proj, data.edge_index, data.train_mask)
        
        loss.backward()
        optimizer.step()
        
        # Validation and testing
        with torch.no_grad():
            model.eval()
            logits, _ = model(data.x, data.edge_index)
            pred = logits.argmax(dim=1)
            
            # Calculate validation F1
            val_f1 = f1_score(data.y[data.val_mask].cpu(), 
                            pred[data.val_mask].cpu(),
                            average='macro')
            
            # Early stopping check
            if val_f1 > best_val_f1:
                best_val_f1 = val_f1
                best_model = model.state_dict()
                # Calculate test F1
                test_f1 = f1_score(data.y[data.test_mask].cpu(),
                                 pred[data.test_mask].cpu(),
                                 average='macro')
    
    # Load best model
    model.load_state_dict(best_model)
    return test_f1

In [ ]:
results = {ratio: [] for ratio in training_ratios}
for ratio in training_ratios:
    print(f"\n=== Training Ratio {ratio*100}% ===")
    for run in range(num_runs):
        test_f1 = train_model(data, run, ratio)
        results[ratio].append(test_f1)
        print(f"Run {run+1}: Test Macro-F1 = {test_f1:.4f}")

# Final reporting
print("\nFinal Results (Macro-F1 ± Std):")
for ratio in training_ratios:
    f1_scores = results[ratio]
    mean = np.mean(f1_scores)
    std = np.std(f1_scores)
    print(f"{ratio*100}% Training: {mean:.2f} ± {std:.2f}")

NameError: name 'training_ratios' is not defined